## Checkwx API (Last updated: 4/27)

## Set up API Request

In [1]:
# quick setup
api = 'https://api.aviationstack.com/v1/flights?access_key='
key = '4ce1a691897a22b88b95e21247d6c83a'

# flight date
# f_d = "2022-04-19"
# from where to where
dep_icao = ['KJFK', 'KEWR', 'KLGA']
# arr_icao = 

In [22]:
import requests, json
import pandas as pd
from pymongo import MongoClient
import psycopg2

## Make a List of Icaos


In [3]:
icaos = ['SPI', 'ABQ', 'LTS', 'AVL', 'AGS', 'AUS', 'GRB', 'BWI', 'BGR', 'BAD', 'BAB', 'LIT', 'BIL', 'BHM', 'LEX', 'BFI', 'BOI', 'BDL', 'BUF', 'CVS', 'BMI', 'CHS', 'CLT', 'MDW', 'ORD', 'RFD', 'CVG', 'COS', 'CLE', 'CAE', 'CBM', 'CRP', 'DFW', 'DAL', 'MSN', 'HNL', 'DAB', 'DEN', 'DSM', 'VPS', 'DTW', 'DLF', 'MGE', 'DOV', 'DBQ', 'DLH', 'DYS', 'EDW', 'OMA', 'ERI', 'FAI', 'SKA', 'FLL', 'FSM', 'FWA', 'AFW', 'FTW', 'BOS', 'MKE', 'PIA', 'IAH', 'ROC', 'GSP', 'GUS', 'GPT', 'ATL', 'HMN', 'HSV', 'IND', 'JAN', 'JAX', 'DAY', 'JFK', 'CMH', 'SNA', 'ADW', 'JLN', 'MCI', 'LGA', 'LFT', 'LFI', 'LAX', 'MSY', 'SDF', 'CHA', 'LBB', 'LUF', 'MCF', 'MHT', 'MBS', 'LAS', 'TCM', 'TYS', 'MEM', 'OAK', 'MIA', 'MSP', 'MOB', 'MLU', 'MGM', 'MUO', 'BNA', 'EWR', 'PHF', 'ORF', 'SJC', 'ONT', 'MCO', 'SFB', 'PBI', 'PHL', 'PHX', 'GSO', 'PIT', 'PDX', 'PWM', 'MLI', 'RDU', 'RND', 'HIB', 'RNO', 'RIC', 'AMA', 'LCK', 'ROA', 'WRB', 'RST', 'DCA', 'SMF', 'SLC', 'SAT', 'SAN', 'SFO', 'SRQ', 'SAV', 'BLV', 'SEA', 'GSB', 'SSC', 'SPS', 'SUX', 'SBN', 'RSW', 'SUS', 'GEG', 'SGF', 'STL', 'SYR', 'TLH', 'TPA', 'ANC', 'CID', 'PVD', 'TIK', 'TOL', 'SUU', 'TRI', 'HTS', 'TUS', 'TUL', 'PAM', 'END', 'VBG', 'IAD', 'SZL', 'ICT', 'OKC', 'HOU', 'FFO']

## API  Setup

In [4]:
hdr = "?x-api-key=81701c75782e420185411f33ec"

url = "https://api.checkwx.com/metar/K"

icao = 'SFO'

resp1 = requests.get(url+icao+'/decoded'+hdr).json()

In [5]:
type(resp1)

dict

In [6]:
# a not decoded response
# kind of confused
print('If you understand this simple output, it will be easy for you to create a dataframe')
print('Only when this short version is meaningful\n')
print(resp1['data'])

If you understand this simple output, it will be easy for you to create a dataframe
Only when this short version is meaningful

[{'barometer': {'hg': 30.11, 'hpa': 1020.0, 'kpa': 101.96, 'mb': 1019.62}, 'clouds': [{'base_feet_agl': 2300, 'base_meters_agl': 701, 'code': 'FEW', 'feet': 2300, 'meters': 701, 'text': 'Few'}, {'base_feet_agl': 3300, 'base_meters_agl': 1006, 'code': 'FEW', 'feet': 3300, 'meters': 1006, 'text': 'Few'}], 'dewpoint': {'celsius': 4, 'fahrenheit': 39}, 'elevation': {'feet': 7, 'meters': 2}, 'flight_category': 'VFR', 'humidity': {'percent': 45}, 'icao': 'KSFO', 'observed': '2022-04-28T21:56Z', 'raw_text': 'KSFO 282156Z 29027G48KT 10SM FEW023 FEW033 16/04 A3011 RMK AO2 PK WND 30048/2154 SLP196 T01560044', 'station': {'geometry': {'coordinates': [-122.375, 37.618999], 'type': 'Point'}, 'location': 'San Francisco, CA, USA', 'name': 'San Francisco International Airport', 'type': 'Airport'}, 'temperature': {'celsius': 16, 'fahrenheit': 61}, 'visibility': {'meters': '16,

## Setup Pymongo and Store data into MongoDB

In [7]:
client = MongoClient()
# Getting a Database
db = client['1234']
# Getting a Collection
collection = db['1234']

## Insert

In [8]:
weather = []

for i in icaos:
    # Extract from Checkwx API 
    resp = requests.get(url+i+'/decoded'+hdr).json()
    weather.append(resp)
    
print(len(weather))

169


In [9]:
for i in range(0,len(weather)):
    # exclude none values
    if weather[i]['data'] != []:
        collection.insert_many(weather[i]['data'])
        
# number of documents in the collection
total_count = collection.count_documents({})
print("Total number of documents : ", total_count)

Total number of documents :  332


## Double check the dataset

In [11]:
# find name's value in location
print(collection.find_one({"station.name": "Wright-Patterson Air Force Base"}))

{'_id': ObjectId('626703681ef0f560a9b94e10'), 'barometer': {'hg': 29.98, 'hpa': 1015.0, 'kpa': 101.52, 'mb': 1015.22}, 'ceiling': {'base_feet_agl': 3500, 'base_meters_agl': 1067, 'code': 'BKN', 'feet': 3500, 'meters': 1067, 'text': 'Broken'}, 'clouds': [{'base_feet_agl': 400, 'base_meters_agl': 122, 'code': 'FEW', 'feet': 400, 'meters': 122, 'text': 'Few'}, {'base_feet_agl': 2000, 'base_meters_agl': 610, 'code': 'SCT', 'feet': 2000, 'meters': 610, 'text': 'Scattered'}, {'base_feet_agl': 3500, 'base_meters_agl': 1067, 'code': 'BKN', 'feet': 3500, 'meters': 1067, 'text': 'Broken'}, {'base_feet_agl': 4200, 'base_meters_agl': 1280, 'code': 'OVC', 'feet': 4200, 'meters': 1280, 'text': 'Overcast'}], 'conditions': [{'code': 'RA', 'prefix': '-', 'text': 'Light Rain'}], 'dewpoint': {'celsius': 17, 'fahrenheit': 63}, 'elevation': {'feet': 820, 'meters': 250}, 'flight_category': 'MVFR', 'humidity': {'percent': 88}, 'icao': 'KFFO', 'observed': '2022-04-25T19:50Z', 'raw_text': 'KFFO 251950Z 28014G2

In [12]:
# find hd's value in barometer
print(collection.find_one({"barometer.hg": 30.28}))

{'_id': ObjectId('626703681ef0f560a9b94d9a'), 'barometer': {'hg': 30.28, 'hpa': 1025.0, 'kpa': 102.54, 'mb': 1025.42}, 'clouds': [{'base_feet_agl': 5000, 'base_meters_agl': 1524, 'code': 'SCT', 'feet': 5000, 'meters': 1524, 'text': 'Scattered'}, {'base_feet_agl': 25000, 'base_meters_agl': 7620, 'code': 'SCT', 'feet': 25000, 'meters': 7620, 'text': 'Scattered'}], 'dewpoint': {'celsius': -3, 'fahrenheit': 27}, 'elevation': {'feet': 981, 'meters': 299}, 'flight_category': 'VFR', 'humidity': {'percent': 35}, 'icao': 'KOMA', 'observed': '2022-04-25T19:52Z', 'raw_text': 'KOMA 251952Z 32015G26KT 10SM SCT050 SCT250 12/M03 A3028 RMK AO2 PK WND 33031/1920 SLP257 T01171033', 'station': {'geometry': {'coordinates': [-95.894096, 41.3032], 'type': 'Point'}, 'location': 'Omaha, NE, USA', 'name': 'Eppley Airfield', 'type': 'Airport'}, 'temperature': {'celsius': 12, 'fahrenheit': 54}, 'visibility': {'meters': '16,100', 'meters_float': 16100.0, 'miles': '10', 'miles_float': 10.0}, 'wind': {'degrees': 32

## Extract Flight Data From API

In [13]:
# quick setup
api = 'https://api.aviationstack.com/v1/flights?access_key='
key = '957d6a3b264de3812c9b69a8a82a014b'

# flight date
# f_d = "2022-04-19"
# from where to where
dep_icao = ['KJFK', 'KEWR', 'KLGA']
# arr_icao = 

In [14]:
# (re)define a df
df = pd.DataFrame()

# give unique index to every flight
flight_index = 'RTF'
count = 0

# get all the flights depart from JFK or EWR
for i in dep_icao:
    for r in icaos:
      # edit url to be relavent to our conrerns
      url = api+key+"&dep_icao="+i+"&arr_icao=K"+r
      # access api results
      res_avi = requests.get(url).json() # list

      # extract variables from api data
      for n in res_avi['data']:
        # ------ flight info -------
        flight_date = n['flight_date']
        flight_status = n['flight_status']
        delay = n['arrival']['delay']

        # ------ location info -------
        # departure place
        departure = n['departure']['airport']
        dicao = n['departure']['icao']
        d_delay = n['departure']['delay']
        d_scheduled = n['departure']['scheduled']
        d_estimated = n['departure']['estimated']
        d_actual = n['departure']['actual']
        d_timezone = n['departure']['timezone']
        d_terminal = n['departure']['terminal']
        d_gate = n['departure']['gate']
        # arrival place
        arrival = n['arrival']['airport']
        aicao = n['arrival']['icao']
        a_scheduled = n['arrival']['scheduled']
        a_estimated = n['arrival']['estimated']
        a_actual = n['arrival']['actual']
        a_timezone = n['arrival']['timezone']
        a_terminal = n['arrival']['terminal']
        a_gate = n['arrival']['gate']
        a_baggage = n['arrival']['baggage']

        # ------ airline info -------
        airline_company = n['airline']['name']

        # count + 1 for every read
        count += 1

        # save variables into a dataframe
        df = df.append({
            # index
              'flight_id': (flight_index+str(count)),
            # flight info
              'flight_date':flight_date, 
              'delay':delay,
              'flight_status':flight_status,
              'airline_company':airline_company,
            # trip infor
              'departure':departure, 'arrival':arrival,
              'dicao':dicao, 'aicao':aicao,
              'd_timezone':d_timezone, 'a_timezone':a_timezone,
              'd_gate':d_gate, 'a_gate':a_gate, 
              'd_terminal':d_terminal, 'a_terminal':a_terminal,
              'a_baggage':a_baggage,
              'd_scheduled':d_scheduled, 'a_scheduled':a_scheduled,
              'd_estimated':d_estimated, 'a_estimated':a_estimated,
              'd_actual':d_actual, 'a_actual':a_actual},
              ignore_index = True)


## Inspect df

In [15]:
df.shape

(18598, 22)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18598 entries, 0 to 18597
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   flight_id        18598 non-null  object
 1   flight_date      18598 non-null  object
 2   delay            5513 non-null   object
 3   flight_status    18592 non-null  object
 4   airline_company  18598 non-null  object
 5   departure        18598 non-null  object
 6   arrival          18597 non-null  object
 7   dicao            18598 non-null  object
 8   aicao            18598 non-null  object
 9   d_timezone       18598 non-null  object
 10  a_timezone       18597 non-null  object
 11  d_gate           17020 non-null  object
 12  a_gate           16433 non-null  object
 13  d_terminal       14236 non-null  object
 14  a_terminal       17321 non-null  object
 15  a_baggage        7329 non-null   object
 16  d_scheduled      18598 non-null  object
 17  a_scheduled      18598 non-null

In [17]:
# check na
df.isnull().sum()[df.isnull().sum()>0]

delay            13085
flight_status        6
arrival              1
a_timezone           1
d_gate            1578
a_gate            2165
d_terminal        4362
a_terminal        1277
a_baggage        11269
d_actual          2040
a_actual          4800
dtype: int64

In [18]:
df.head()

,flight_id,flight_date,delay,flight_status,airline_company,departure,arrival,dicao,aicao,d_timezone,...,a_gate,d_terminal,a_terminal,a_baggage,d_scheduled,a_scheduled,d_estimated,a_estimated,d_actual,a_actual
0,RTF1,2022-04-28,None,scheduled,JetBlue Airways,John F Kennedy International,Albuquerque International,KJFK,KABQ,America/New_York,...,None,5,None,None,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,None,None
1,RTF2,2022-04-28,None,scheduled,Qatar Airways,John F Kennedy International,Albuquerque International,KJFK,KABQ,America/New_York,...,None,5,None,None,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,None,None
2,RTF3,2022-04-28,None,scheduled,Etihad Airways,John F Kennedy International,Albuquerque International,KJFK,KABQ,America/New_York,...,None,5,None,None,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,None,None
3,RTF4,2022-04-28,None,scheduled,Singapore Airlines,John F Kennedy International,Albuquerque International,KJFK,KABQ,America/New_York,...,None,5,None,None,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,None,None
4,RTF5,2022-04-28,None,scheduled,Emirates,John F Kennedy International,Albuquerque International,KJFK,KABQ,America/New_York,...,None,5,None,None,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,2022-04-28T19:30:00+00:00,2022-04-28T22:43:00+00:00,None,None


In [19]:
len(df)

18598

### Let's see how many flights are in each airport

In [20]:
print(df['departure'].value_counts())

Newark Liberty International    6551
John F Kennedy International    6032
La Guardia                      6015
Name: departure, dtype: int64


## SQL Database Setup


In [23]:
print('Successfully Connected to the PostgreSQL database!Yeah!!🎉🎉🎉')

# colab cannot connect to localhost so plz
# switch to Jupyter Notebook
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    # name of database
    # you need to have the same one
    database="project",
    user="postgres",
    password="123")

Successfully Connected to the PostgreSQL database!Yeah!!🎉🎉🎉


In [24]:
# create a cursor
cur = conn.cursor()
cur.execute('rollback;')

In [25]:
# clear all tables before creating tables
cur.execute('DROP TABLE IF EXISTS flight CASCADE;')
cur.execute('DROP TABLE IF EXISTS trip CASCADE;')
cur.execute('DROP TABLE IF EXISTS departure CASCADE;')
cur.execute('DROP TABLE IF EXISTS arrival CASCADE;')

## Create Relational Tables


In [26]:
create_flight = """ CREATE TABLE flight (
                            flight_id varchar(15) NOT NULL,
                            flight_date varchar(50),
                            flight_status varchar(10),
                            delay int,
                            airline_company varchar(50),
                            PRIMARY KEY (flight_id)
                            )"""

create_trip = """ CREATE TABLE trip (
                            flight_id varchar(15) NOT NULL,
                            dicao varchar(5),
                            aicao varchar(5),
                            PRIMARY KEY (flight_id),
                            FOREIGN KEY (flight_id) REFERENCES flight (flight_id)
                            )"""

create_departure = """ CREATE TABLE departure (
                            flight_id varchar(15) NOT NULL,
                            departure varchar(100),
                            dicao varchar(5),
                            d_terminal varchar(6),
                            d_gate varchar(5),
                            d_scheduled varchar(50),
                            d_estimated varchar(50),
                            d_actual varchar(50),
                            PRIMARY KEY (flight_id),
                            FOREIGN KEY (flight_id) REFERENCES trip (flight_id)
                            )"""

create_arrival = """ CREATE TABLE arrival (
                            flight_id varchar(15) NOT NULL,
                            arrival varchar(100),
                            aicao varchar(5),
                            a_baggage varchar(10),
                            a_terminal varchar(6),
                            a_gate varchar(5),
                            a_scheduled varchar(50),
                            a_estimated varchar(50),
                            a_actual varchar(50),
                            PRIMARY KEY (flight_id),
                            FOREIGN KEY (flight_id) REFERENCES trip (flight_id)
                            )"""


cur.execute(create_flight)
cur.execute(create_trip)
cur.execute(create_departure)
cur.execute(create_arrival)

conn.commit()

In [27]:
cur.execute('rollback;')

## Insert Data into SQL database


In [28]:
def insert_into_table(curr,
                      # table 1
                      flight_id, flight_date, flight_status, delay, airline_company,
                      # table 2
                      dicao, aicao,
                      # table 3
                      departure, d_terminal, d_gate,
                      d_scheduled, d_estimated, d_actual,
                      # table 4
                      arrival, a_terminal, a_gate,
                      a_baggage, a_scheduled, a_estimated, a_actual):
    
    
    # table 1
    insert_into_flight = ("""INSERT INTO flight (flight_id, flight_date, flight_status,
                                                 delay, airline_company)
                                                 VALUES(%s,%s,%s,%s,%s);""")
    row_to_insert1 = (flight_id, flight_date, flight_status, delay, airline_company)
    curr.execute(insert_into_flight, row_to_insert1)
    
    
    # table 2
    insert_into_trip = ("""INSERT INTO trip (flight_id, dicao, aicao)
                                                 VALUES(%s,%s,%s);""")
    row_to_insert2 = (flight_id, dicao, aicao)
    curr.execute(insert_into_trip, row_to_insert2)
    
    
    # table 3
    insert_into_departure = ("""INSERT INTO departure (flight_id, departure, dicao, d_terminal, d_gate,
                                                       d_scheduled, d_estimated, d_actual)
                                                       VALUES(%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert3 = (flight_id, departure, dicao, d_terminal, d_gate, d_scheduled, d_estimated, d_actual)
    curr.execute(insert_into_departure, row_to_insert3)
    
    
    # table 4
    insert_into_arrival = ("""INSERT INTO arrival (flight_id, arrival, aicao, a_terminal, a_gate,
                                                      a_baggage, a_scheduled, a_estimated, a_actual)
                                                       VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert4 = (flight_id, arrival, aicao, a_terminal, a_gate, a_baggage, a_scheduled, a_estimated, a_actual)
    curr.execute(insert_into_arrival, row_to_insert4)


In [29]:
def append_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(cur, 
                          row['flight_id'], row['flight_date'], row['flight_status'], row['delay'], 
                          row['airline_company'], row['dicao'], row['aicao'], row['departure'], row['d_terminal'], row['d_gate'], 
                          row['d_scheduled'], row['d_estimated'], row['d_actual'], row['arrival'], row['a_terminal'],
                          row['a_gate'], row['a_baggage'], row['a_scheduled'], row['a_estimated'], row['a_actual'])

In [30]:
# insert values into db tables
append_from_df_to_db(cur, df)
conn.commit()

In [31]:
# contract: 
cur.execute(""" SELECT * FROM flight LIMIT 10 """)

d_rows = cur.fetchall()
colnames_d = [desc[0] for desc in cur.description]
df_d = pd.DataFrame(d_rows, columns = colnames_d)
print(df_d)

  flight_id flight_date flight_status delay     airline_company
0      RTF1  2022-04-28     scheduled  None     JetBlue Airways
1      RTF2  2022-04-28     scheduled  None       Qatar Airways
2      RTF3  2022-04-28     scheduled  None      Etihad Airways
3      RTF4  2022-04-28     scheduled  None  Singapore Airlines
4      RTF5  2022-04-28     scheduled  None            Emirates
5      RTF6  2022-04-27        landed  None               El Al
6      RTF7  2022-04-27        landed  None     JetBlue Airways
7      RTF8  2022-04-27        landed  None      Etihad Airways
8      RTF9  2022-04-27        landed  None       Qatar Airways
9     RTF10  2022-04-27        landed  None  Singapore Airlines


In [32]:
# contract: 
cur.execute(""" SELECT * FROM trip LIMIT 10 """)

d_rows = cur.fetchall()
colnames_d = [desc[0] for desc in cur.description]
df_d = pd.DataFrame(d_rows, columns = colnames_d)
print(df_d)

  flight_id dicao aicao
0      RTF1  KJFK  KABQ
1      RTF2  KJFK  KABQ
2      RTF3  KJFK  KABQ
3      RTF4  KJFK  KABQ
4      RTF5  KJFK  KABQ
5      RTF6  KJFK  KABQ
6      RTF7  KJFK  KABQ
7      RTF8  KJFK  KABQ
8      RTF9  KJFK  KABQ
9     RTF10  KJFK  KABQ


In [33]:
# contract: 
cur.execute(""" SELECT * FROM departure LIMIT 10 """)

d_rows = cur.fetchall()
colnames_d = [desc[0] for desc in cur.description]
df_d = pd.DataFrame(d_rows, columns = colnames_d)
print(df_d)

  flight_id                     departure dicao d_terminal d_gate  \
0      RTF1  John F Kennedy International  KJFK          5     16   
1      RTF2  John F Kennedy International  KJFK          5     16   
2      RTF3  John F Kennedy International  KJFK          5     16   
3      RTF4  John F Kennedy International  KJFK          5     16   
4      RTF5  John F Kennedy International  KJFK          5     16   
5      RTF6  John F Kennedy International  KJFK          5      9   
6      RTF7  John F Kennedy International  KJFK          5      9   
7      RTF8  John F Kennedy International  KJFK          5      9   
8      RTF9  John F Kennedy International  KJFK          5      9   
9     RTF10  John F Kennedy International  KJFK          5      9   

                 d_scheduled                d_estimated  \
0  2022-04-28T19:30:00+00:00  2022-04-28T19:30:00+00:00   
1  2022-04-28T19:30:00+00:00  2022-04-28T19:30:00+00:00   
2  2022-04-28T19:30:00+00:00  2022-04-28T19:30:00+00:00   
3  2

In [34]:
# contract: 
cur.execute(""" SELECT * FROM arrival LIMIT 10 """)

d_rows = cur.fetchall()
colnames_d = [desc[0] for desc in cur.description]
df_d = pd.DataFrame(d_rows, columns = colnames_d)
print(df_d)

  flight_id                    arrival aicao a_baggage a_terminal a_gate  \
0      RTF1  Albuquerque International  KABQ      None       None   None   
1      RTF2  Albuquerque International  KABQ      None       None   None   
2      RTF3  Albuquerque International  KABQ      None       None   None   
3      RTF4  Albuquerque International  KABQ      None       None   None   
4      RTF5  Albuquerque International  KABQ      None       None   None   
5      RTF6  Albuquerque International  KABQ      None          4   None   
6      RTF7  Albuquerque International  KABQ      None          4   None   
7      RTF8  Albuquerque International  KABQ      None          4   None   
8      RTF9  Albuquerque International  KABQ      None          4   None   
9     RTF10  Albuquerque International  KABQ      None          4   None   

                 a_scheduled                a_estimated  \
0  2022-04-28T22:43:00+00:00  2022-04-28T22:43:00+00:00   
1  2022-04-28T22:43:00+00:00  2022-04-28T22:4

## Join data from different database

In [35]:
import pymongo
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
db = client['1234']
collection = db['1234']
df_checkwx = pd.DataFrame(list(collection.find()))

In [45]:
# Select the column we needs
checkwx =  df_checkwx[['icao','observed', 'temperature','rain','humidity','clouds', 'elevation','visibility','wind']]

In [38]:
# Convert observed time from object type to timestamp type
checkwx['observed']=pd.to_datetime(checkwx['observed'], format='%Y/%m/%d %H:%M:%S')
# Only keep the latest data
checkwx = checkwx.drop_duplicates(subset=['icao'], keep='last')


C:\Users\sayua\AppData\Local\Temp/ipykernel_15500/3321387330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkwx['observed']=pd.to_datetime(checkwx['observed'], format='%Y/%m/%d %H:%M:%S')


In [39]:
# Check if data contains duplicate
checkwx['icao'].value_counts()

KSPI    1
KONT    1
KSFB    1
KPBI    1
KPHL    1
       ..
KMKE    1
KPIA    1
KIAH    1
KROC    1
KFFO    1
Name: icao, Length: 166, dtype: int64

In [46]:
# Rename the weather data for front-end
checkwx_rename = checkwx.rename(columns={'icao': 'Arrival Airport ID','observed': 'Weather observed time','temperature': 'Temperature',
                              'rain': 'Rain','humidity': 'Humidity','clouds': 'Clouds',
                              'elevation': 'Elevation','visibility': 'Visibility','wind': 'Wind'})

In [47]:
# Rename the flight data for front-end
df_flight =df[['aicao','dicao','airline_company','a_scheduled','a_terminal','a_gate','a_baggage','arrival', 'd_scheduled','d_gate','d_terminal','delay','departure','flight_id','flight_status']]
df_flight_rename = df_flight.rename(columns={'airline_company': 'Airline','a_scheduled': 'Arriving scheduled time','a_terminal': 'Arriving terminal',
                              'a_gate': 'Arriving gate','a_baggage': 'baggage line','arrival': 'Arrival airport',
                              'd_scheduled': 'Departure scheduled time','d_gate': 'Departure gate','d_terminal': 'Departure terminal','departure': 'Departure airpoty',
                              'flight_id': 'Flight ID','flight_status': 'Flight Status','delay': 'Delay',
                                            'aicao': 'Arrival Airport ID','dicao': 'Departure Airport ID'})

In [48]:
# Review all the columns
pd.set_option('display.max_columns', 500)

# Inner join two dataset based on icao
F_end = pd.merge(df_flight_rename, checkwx_rename, how='inner', on=['Arrival Airport ID'])
F_end.head()

,Arrival Airport ID,Departure Airport ID,Airline,Arriving scheduled time,Arriving terminal,Arriving gate,baggage line,Arrival airport,Departure scheduled time,Departure gate,Departure terminal,Delay,Departure airpoty,Flight ID,Flight Status,Weather observed time,Temperature,Rain,Humidity,Clouds,Elevation,Visibility,Wind
0,KABQ,KJFK,JetBlue Airways,2022-04-28T22:43:00+00:00,None,None,None,Albuquerque International,2022-04-28T19:30:00+00:00,16,5,None,John F Kennedy International,RTF1,scheduled,2022-04-25T19:52Z,"{'celsius': 17, 'fahrenheit': 63}",NaN,{'percent': 19},"[{'base_feet_agl': 9000, 'base_meters_agl': 27...","{'feet': 5315, 'meters': 1620}","{'meters': '16,100', 'meters_float': 16100.0, ...","{'degrees': 0, 'speed_kph': 7, 'speed_kts': 4,..."
1,KABQ,KJFK,JetBlue Airways,2022-04-28T22:43:00+00:00,None,None,None,Albuquerque International,2022-04-28T19:30:00+00:00,16,5,None,John F Kennedy International,RTF1,scheduled,2022-04-28T22:25Z,"{'celsius': 26, 'fahrenheit': 79}",NaN,{'percent': 7},"[{'base_feet_agl': 25000, 'base_meters_agl': 7...","{'feet': 5315, 'meters': 1620}","{'meters': '16,100', 'meters_float': 16100.0, ...","{'degrees': 180, 'gust_kph': 33, 'gust_kts': 1..."
2,KABQ,KJFK,Qatar Airways,2022-04-28T22:43:00+00:00,None,None,None,Albuquerque International,2022-04-28T19:30:00+00:00,16,5,None,John F Kennedy International,RTF2,scheduled,2022-04-25T19:52Z,"{'celsius': 17, 'fahrenheit': 63}",NaN,{'percent': 19},"[{'base_feet_agl': 9000, 'base_meters_agl': 27...","{'feet': 5315, 'meters': 1620}","{'meters': '16,100', 'meters_float': 16100.0, ...","{'degrees': 0, 'speed_kph': 7, 'speed_kts': 4,..."
3,KABQ,KJFK,Qatar Airways,2022-04-28T22:43:00+00:00,None,None,None,Albuquerque International,2022-04-28T19:30:00+00:00,16,5,None,John F Kennedy International,RTF2,scheduled,2022-04-28T22:25Z,"{'celsius': 26, 'fahrenheit': 79}",NaN,{'percent': 7},"[{'base_feet_agl': 25000, 'base_meters_agl': 7...","{'feet': 5315, 'meters': 1620}","{'meters': '16,100', 'meters_float': 16100.0, ...","{'degrees': 180, 'gust_kph': 33, 'gust_kts': 1..."
4,KABQ,KJFK,Etihad Airways,2022-04-28T22:43:00+00:00,None,None,None,Albuquerque International,2022-04-28T19:30:00+00:00,16,5,None,John F Kennedy International,RTF3,scheduled,2022-04-25T19:52Z,"{'celsius': 17, 'fahrenheit': 63}",NaN,{'percent': 19},"[{'base_feet_agl': 9000, 'base_meters_agl': 27...","{'feet': 5315, 'meters': 1620}","{'meters': '16,100', 'meters_float': 16100.0, ...","{'degrees': 0, 'speed_kph': 7, 'speed_kts': 4,..."


## Data Storing

In [44]:
F_end.to_csv('Front end dataset.csv')